In [1]:
import os 
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

In [2]:
!kaggle datasets download -d rahilmehtaucoe2784/indian-currency

 99% 85.0M/85.7M [00:03<00:00, 14.7MB/s]
100% 85.7M/85.7M [00:03<00:00, 27.0MB/s]


In [3]:
%%capture
!unzip \*.zip && rm *.zip &> /dev/null

In [4]:
!pip install -q tflite-model-maker
!pip install -q pycocotools



     |████████████████████████████████| 616 kB 3.9 MB/s 
     |████████████████████████████████| 120 kB 68.0 MB/s 
     |████████████████████████████████| 1.1 MB 62.7 MB/s 
     |████████████████████████████████| 234 kB 51.9 MB/s 
     |████████████████████████████████| 840 kB 58.9 MB/s 
     |████████████████████████████████| 3.4 MB 57.5 MB/s 
     |████████████████████████████████| 1.2 MB 28.1 MB/s 
     |████████████████████████████████| 596 kB 72.1 MB/s 
     |████████████████████████████████| 6.4 MB 58.7 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 87 kB 7.5 MB/s 
     |████████████████████████████████| 1.1 MB 57.5 MB/s 
     |████████████████████████████████| 25.3 MB 1.2 MB/s 
     |████████████████████████████████| 99 kB 12.9 MB/s 
     |████████████████████████████████| 352 kB 66.2 MB/s 
     |████████████████████████████████| 47.7 MB 1.6 MB/s 
     |████████████████████████████████| 462 kB 74.0 MB/s 
     |██████████████

In [5]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
import time

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
from tensorflow import keras

In [6]:
train_image_dir = '/content/final_currency_dataset/train images'
train_annotation_dir = '/content/final_currency_dataset/train anotation'
validate_image_dir = '/content/final_currency_dataset/validate Images'
validate_anaotation_dir = '/content/final_currency_dataset/validate anotation '
test_image_dir = '/content/final_currency_dataset/test image'
test_anotation_dir = '/content/final_currency_dataset/test anotation'

In [7]:
train_dir = object_detector.DataLoader.from_pascal_voc(train_image_dir, train_annotation_dir, label_map={1:"10", 2: "20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [8]:
validate_dir = object_detector.DataLoader.from_pascal_voc(validate_image_dir, validate_anaotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [9]:
test_dir = object_detector.DataLoader.from_pascal_voc(test_image_dir, test_anotation_dir, label_map={1:"10", 2:"20",3:"50",4:"100",5:"200",6:"500",7:"2000"})

In [10]:
print(len(train_dir))
print(len(validate_dir))
print(len(test_dir))

2383
671
345


In [11]:
spec = model_spec.get('efficientdet_lite0')

In [12]:
start = time.time()
model = object_detector.create(train_dir, 
                               model_spec=spec, 
                               epochs=10, 
                               batch_size=6, 
                               train_whole_model=True, 
                               validation_data=validate_dir)
end = time.time()

Epoch 1/10
397/397 [==============================] - 111s 188ms/step - det_loss: 1.0581 - cls_loss: 0.7907 - box_loss: 0.0053 - reg_l2_loss: 0.0641 - loss: 1.1221 - learning_rate: 0.0078 - gradient_norm: 3.0024 - val_det_loss: 0.7302 - val_cls_loss: 0.6195 - val_box_loss: 0.0022 - val_reg_l2_loss: 0.0643 - val_loss: 0.7944
Epoch 2/10
397/397 [==============================] - 73s 183ms/step - det_loss: 0.6764 - cls_loss: 0.5353 - box_loss: 0.0028 - reg_l2_loss: 0.0644 - loss: 0.7409 - learning_rate: 0.0070 - gradient_norm: 3.0447 - val_det_loss: 0.5310 - val_cls_loss: 0.4538 - val_box_loss: 0.0015 - val_reg_l2_loss: 0.0646 - val_loss: 0.5956
Epoch 3/10
397/397 [==============================] - 72s 183ms/step - det_loss: 0.5951 - cls_loss: 0.4814 - box_loss: 0.0023 - reg_l2_loss: 0.0647 - loss: 0.6598 - learning_rate: 0.0061 - gradient_norm: 3.0753 - val_det_loss: 0.4257 - val_cls_loss: 0.3544 - val_box_loss: 0.0014 - val_reg_l2_loss: 0.0648 - val_loss: 0.4905
Epoch 4/10
397/397 [====

In [13]:
print("The time of execution of training is :", end-start)

The time of execution of training is : 828.3327279090881


In [14]:
start = time.time()
ap = model.evaluate(test_dir,batch_size=5)
print(ap)
end = time.time()


69/69 [==============================] - 11s 104ms/step

{'AP': 0.6363581, 'AP50': 0.78524816, 'AP75': 0.72951263, 'APs': -1.0, 'APm': 0.42806292, 'APl': 0.6499516, 'ARmax1': 0.6742318, 'ARmax10': 0.7917895, 'ARmax100': 0.79944265, 'ARs': -1.0, 'ARm': 0.55, 'ARl': 0.8132934, 'AP_/10': 0.42366618, 'AP_/20': 0.71495056, 'AP_/50': 0.42770353, 'AP_/100': 0.7387704, 'AP_/200': 0.780183, 'AP_/500': 0.7252539, 'AP_/2000': 0.64397883}


In [15]:
print("The time of execution of evaluation is  :", end-start)

The time of execution of evaluation is  : 23.599231481552124


In [ ]:
#model = object_detector.create(train_dir, model_spec=spec, epochs=30, batch_size=8, train_whole_model=True)

In [16]:
model.export(export_dir='/content/')